In [1]:
import numpy as np
import os
import pandas as pd
import geopandas as gpd
import shapely

In [2]:
class KMeans(object):
    
    def __init__(self, n_clusters = 8, dist = 'Euclid'):
        self.n_clusters = n_clusters
        self.cluster_centers = np.zeros((n_clusters,2))
        if dist == 'Euclid':
            self._distance = self._distance_euclid
        elif dist == 'Geodesic':
            self._distance = self._distance_haversine
        
    
    def _has_converged(self,old_centers, new_centers):
        return np.array_equal(old_centers, new_centers)
    
    def _compute_clusters(self, X):
        cluster_list = np.zeros((len(X)),dtype=np.int)
        for i,x in enumerate(X):
            cluster_list[i] = np.argmin(
                np.array([self._distance(x,self.cluster_centers[k])
                          for k in range(self.n_clusters)]))
        return cluster_list
        
    def _recompute_centers(self, X):
        #centers = self.cluster_centers.copy()
        centers = np.zeros((self.n_clusters,2))
        for k in range(self.n_clusters):
            points = []
            for index, item in enumerate(self.labels):
#                 print(f'Index = {index}')
#                 print(f'Item = {item}')
                if(item == k):
#                     print(f'Point = {X[index]}')
                    points.append(X[index])
            points = np.array(points)
      #      print(f'Mean: {np.mean(points, axis=0)}')
            if(points.size == 0):
                print(self.cluster_centers)
                print(self.n_clusters)
                print(self.labels)
                print(np.mean(points, axis=0))
            centers[k] = np.mean(points, axis=0)
          #  centers.append(np.mean(points,axis=0))
       # print(centers)
        return centers
        
    def _distance_euclid(self, x,y):
        return np.linalg.norm(np.subtract(x,y))
    
    """
    Expects points to be of the form lat,lon
    """
    def _distance_haversine(self,x,y):
        lat_1, lon_1, lat_2, lon_2 = map(np.radians,[x[0],x[1],y[0],y[1]])
        d_lat = lat_2 - lat_1
        d_lon = lon_2 - lon_1
        
        a = np.sin(d_lat/2.0)**2 + np.cos(lat_1)*np.cos(lat_2)*np.sin(d_lon/2)**2
        
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6372.8 * c
        return km
   
    def _initialize_centers(self,X):
        centers = []
        length = len(X)
        for k in range(self.n_clusters):
            index = np.random.randint(0,length)
            while (tuple(X[index]) in set(map(tuple,centers))):
                index = np.random.randint(0,length)
            centers.append(X[index])
        return np.array(centers)


    def fit(self,X):
        self.X = X
        old_centers = self.cluster_centers.copy()
        self.cluster_centers = self._initialize_centers(X)
       # print(f'Initialized Clusters: {self.cluster_centers}')
        while(not self._has_converged(old_centers, self.cluster_centers.copy())):
            old_centers = self.cluster_centers.copy()
            self.labels = self._compute_clusters(X)
            self.cluster_centers = self._recompute_centers(X)
            
    def fit_from_starting_points(self,X,starting_clusters):
        self.X = X
        old_centers = self.cluster_centers.copy()
        self.cluster_centers = starting_clusters
        while(not self._has_converged(old_centers, self.cluster_centers.copy())):
            old_centers = self.cluster_centers.copy()
            self.labels = self._compute_clusters(X)
            self.cluster_centers = self._recompute_centers(X)
            
    def avg_distance(self):
        centers = self.cluster_centers
        k = self.n_clusters
        avg_distance = 0
        for cluster in range(k):
            total = 0
            dist = 0
            for index, point in enumerate(self.labels):
                if (point == cluster):
                    total += 1
                    dist += self._distance(self.X[index], centers[cluster])
            avg_distance += dist / total
        avg_distance = avg_distance / k
        return avg_distance
                
            
    

In [3]:
from shapely.geometry import Point
from geopandas import GeoDataFrame

demographics = gpd.read_file('./census.geoJSON')

def gen_coords(loc):
    data = loc[1:-1].split(',')
    data = list((np.float(data[0]), np.float(data[1])))
    x.append(data[1])
    y.append(data[0])
    return [data[0],data[1]]

def point_similarity(X,geo_labels, euc_labels,k):
    '''For an inputted series of points, geodesic labels, euclidean labels, and k-value
       returns the point-similarity index per geodesic cluster
    '''

    euc_cluster_totals = np.zeros(k,dtype=np.int)
    geo_euc_composition = [np.zeros(k,dtype=np.int)* 1 for i in range(k)]
    
    for index,point in enumerate(geo_labels):
        euc_cluster_totals[euc_labels[index]] += 1
        geo_euc_composition[point][euc_labels[index]] += 1
    
    point_sim = []
    for geo_cluster in range(k):
        sim = 0
        for euc_cluster in range(k):
            matching_points = geo_euc_composition[geo_cluster][euc_cluster]
            euc_percentage = matching_points / euc_cluster_totals[euc_cluster]
            geo_percentage = matching_points / np.sum(geo_euc_composition[geo_cluster])
            sim += euc_percentage * geo_percentage
        point_sim.append(sim)

    return np.array(point_sim)

def minority_probability(X,cluster_number,geo_labels,demographics):
        points = X[geo_labels == cluster_number]
        # geoJSON puts points in Long/Lat order
        # but points are in lat/long earlier
        hull = shapely.geometry.multipoint.MultiPoint([[p[1],p[0]] for p in points]).convex_hull
  
        pop = np.zeros(7)
        for index in range(len(demographics)):
            census_tract = demographics.loc[index,'geometry']
            intersect = hull.intersection(census_tract)
            overlap = intersect.area/census_tract.area
            if (overlap != 0):
                pop = pop + (np.array(demographics.loc[index,['White','Black or African American', 'American Indian and Ala Native',
                   'Asian','Native Hawaiian/other Pac Isl', 'Multiple Race',
                   'Other Race']]) * overlap)
                            
        return (pop[1:]/np.sum(pop)).sum()

def bias_index(X, geo_labels, euc_labels, demographics, k):

    dissimilarity_index = 1 - point_similarity(X,geo_labels,euc_labels,k)
    minority_prob = np.array([minority_probability(X,cluster,geo_labels,demographics) 
                              for cluster in range(k)])
    potential_bias = minority_prob * dissimilarity_index
    return potential_bias.mean()

    
#bias_index(X,kmeans_theft_geodesic.labels,kmeans_theft_euclid.labels,demographics,k)


In [66]:
array = np.zeros(4,dtype=np.int)
array[0] +=1
print(array)

[1 0 0 0]


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoDataFrame
import os
import warnings

def gen_coords(loc):
    data = loc[1:-1].split(',')
    data = list((np.float(data[0]), np.float(data[1])))
    x.append(data[1])
    y.append(data[0])
    return [data[0],data[1]]

def percent_similarity(set_1, set_2):
    count = 0
    for i in range (len(set_1)):
        if set_1[i] == set_2[i]:
            count += 1
    return count / len(set_1)


for file in os.listdir('../data/data_2015/'):
    if(file.endswith('.csv')):
        df = pd.read_csv('../data/data_2015/' + file, sep =';')
        
        x = []
        y = []

        df['Points'] = df['Location'].apply(gen_coords)
        points = [Point(xy) for xy in zip(x,y)]
        crs = {'init': 'epsg:4326'}
        geo_df = GeoDataFrame(df,crs=crs, geometry=points)
        theft_both = geo_df.copy()

        X = np.array(theft_both['Points'])

        for k in range(2,11):
            while(True):
                kmeans_theft_euclid = KMeans(n_clusters = k)
                kmeans_theft_geodesic = KMeans(n_clusters = k, dist = 'Geodesic')
                centers = kmeans_theft_geodesic._initialize_centers(X)

                kmeans_theft_euclid.fit_from_starting_points(X,centers)
                kmeans_theft_geodesic.fit_from_starting_points(X,centers) 
                if(percent_similarity(kmeans_theft_euclid.labels,
                                     kmeans_theft_geodesic.labels) > .50):
                    break

            theft_both.loc[:,'e_cluster' + 'K' + str(k)] = kmeans_theft_euclid.labels.copy()
            theft_both.loc[:,'g_cluster' + 'K' + str(k)] = kmeans_theft_geodesic.labels.copy()

                
            print(percent_similarity(kmeans_theft_euclid.labels,
                                     kmeans_theft_geodesic.labels))

            
            
        theft_both = theft_both.drop('Points', axis=1)

        try:
            os.remove('./datamound/'+file.split('.csv')[0] + '.js')
        except FileNotFoundError:
            pass
        
        theft_both.to_file('./datamound/'+file.split('.csv')[0] + '.js', driver='GeoJSON')
#         with open('./datamound/'+file.split('.csv')[0] + '.js', 'r') as original: data = original.read()
#         with open('./datamound/'+file.split('.csv')[0] + '.js', 'w') as modified: modified.write('var both =' 
#                                                         + data +';')
        print(file)
        print('-------')


In [ ]:
import os
ordered_names = ['theft_jan.js','theft_feb.js','theft_mar.js','theft_april.js',
                'theft_may.js','theft_june.js','theft_july.js','theft_aug.js',
                'theft_sep.js','theft_oct.js','theft_nov.js','theft_dec.js',
                'm_theft_jan.js','m_theft_feb.js','m_theft_mar.js','m_theft_april.js',
                'm_theft_may.js','m_theft_june.js','m_theft_july.js','m_theft_aug.js',
                'm_theft_sep.js','m_theft_oct.js','m_theft_nov.js','m_theft_dec.js']
data = 'var complete_data =['
for file in ordered_names:
    reader = open('./datamound/' + file,'r')
    data += (reader.read() + ',')
    reader.close()
    print(file)
        
writer = open('all.js','w')
writer.write(data + '];')
writer.close()